In [1]:
import os
from pyiron_base import Project

In [2]:
# input parameter 
domain_size = 2.0

In [3]:
# create pyiron project 
pr = Project("workflow")
pr.remove_jobs(recursive=True, silently=True)

0it [00:00, ?it/s]

In [4]:
# create conda environments for the proprocessing, processing and postprocessing stage 
for k, v in {
    "preprocessing": "source/envs/preprocessing.yaml",
    "processing": "source/envs/processing.yaml",
    "postprocessing": "source/envs/postprocessing.yaml"
}.items():
    pr.conda_environment.create(env_name=k, env_file=v, global_installation=False, use_mamba=True)



==> WARNING: A newer version of conda exists. <==
    current version: 24.3.0
    latest version: 24.5.0

Please update conda by running

    $ conda update -n base -c conda-forge conda




==> WARNING: A newer version of conda exists. <==
    current version: 24.3.0
    latest version: 24.5.0

Please update conda by running

    $ conda update -n base -c conda-forge conda




==> WARNING: A newer version of conda exists. <==
    current version: 24.3.0
    latest version: 24.5.0

Please update conda by running

    $ conda update -n base -c conda-forge conda




In [5]:
# Preprocessing
## generate mesh
gmsh = pr.wrap_executable(
    job_name="gmsh",
    executable_str=f"gmsh -2 -setnumber domain_size {domain_size} unit_square.geo -o square.msh",
    conda_environment_path=pr.conda_environment.preprocessing,
    input_file_lst=["source/unit_square.geo"],
    delayed=True,
    output_file_lst=["square.msh"],
)

In [6]:
## convert mesh to xdmf
meshio = pr.wrap_executable(
    job_name="meshio",
    executable_str="meshio convert square.msh square.xdmf",
    conda_environment_path=pr.conda_environment.preprocessing,
    input_file_lst=[gmsh.files.square_msh],
    delayed=True,
    output_file_lst=["square.xdmf", "square.h5"],
)

In [7]:
# Processing
## poisson
def collect_output(working_directory):
    with open(os.path.join(working_directory, "numdofs.txt"), "r") as f:
        return {"numdofs": int(f.read())}

poisson = pr.wrap_executable(
    job_name="poisson",
    executable_str="python poisson.py --mesh square.xdmf --degree 2 --outputfile poisson.pvd --num-dofs numdofs.txt",
    conda_environment_path=pr.conda_environment.processing,
    input_file_lst=["source/poisson.py", meshio.files.square_xdmf, meshio.files.square_h5],
    delayed=True,
    collect_output_funct=collect_output,
    output_key_lst=["numdofs"],
    output_file_lst=["poisson.pvd", "poisson000000.vtu"],
)

In [8]:
# Postprocessing
## plot over line
pvbatch = pr.wrap_executable(
    job_name="pvbatch",
    executable_str="pvbatch postprocessing.py poisson.pvd plotoverline.csv",
    conda_environment_path=pr.conda_environment.postprocessing,
    input_file_lst=["source/postprocessing.py", poisson.files.poisson_pvd, poisson.files.poisson000000_vtu],
    delayed=True,
    output_file_lst=["plotoverline.csv"],
)

In [9]:
## substitute macros
def write_input(input_dict, working_directory):
    script_name = os.path.join(working_directory, "macros.sh")
    numdofs = input_dict["numdofs"]
    with open(script_name, "w") as f:
        f.writelines(f"python prepare_paper_macros.py --macro-template-file macros.tex.template --plot-data-path plotoverline.csv --domain-size {domain_size} --num-dofs {numdofs} --output-macro-file macros.tex")
    os.chmod(script_name, 0o744)

macros = pr.wrap_executable(
    job_name="macros",
    input_dict={"numdofs": poisson.output.numdofs},
    input_file_lst=["source/macros.tex.template", "source/prepare_paper_macros.py", pvbatch.files.plotoverline_csv],
    write_input_funct=write_input,
    executable_str="./macros.sh",
    delayed=True,
    output_file_lst=["macros.tex"],
)

In [10]:
## compile paper
tectonic = pr.wrap_executable(
    job_name="tectonic",
    executable_str="tectonic paper.tex",
    conda_environment_path=pr.conda_environment.postprocessing,
    input_file_lst=["source/paper.tex", macros.files.macros_tex, pvbatch.files.plotoverline_csv],
    delayed=True,
)

In [11]:
# Execute Workflow Graph
tectonic.pull()

The job gmsh was saved and received the ID: 1
The job meshio was saved and received the ID: 2
The job poisson was saved and received the ID: 3
The job pvbatch was saved and received the ID: 4
The job macros was saved and received the ID: 5
The job tectonic was saved and received the ID: 6


{'groups': ['input', 'storage'], 'nodes': ['DICT_VERSION', 'HDF_VERSION', 'NAME', 'OBJECT', 'TYPE', 'VERSION', 'executable', 'job_id', 'server', 'status']}